###Notes on current discoveries:

-All files in storage claim are accessed with the base address '/nfs/'
-Target files are HTML wrapped in .gz, use gzip to unzip

**possibly try figure out how to get nltk on container**

links:
https://stackoverflow.com/questions/43721142/nltk-corpus-of-gzipped-files
https://www.nltk.org/install.html

In [ ]:
## need to get topics.xml and parse with topics

In [8]:
##import nltk

ModuleNotFoundError: No module named 'nltk'

In [24]:
import gzip
import pandas as pd
from collections import OrderedDict

In [40]:
!!pip install beautifulsoup4
from bs4 import BeautifulSoup as bs

In [210]:
corpus_path = "/nfs/trects-kba2014-filtered" # directory of corpus of gzipped html files
tags = ['streamid', 'docid', 'yyyymmddhh', 'kbastream', 'zulu', 'epoch', 'title', 'text', 'url'] # doc fields
#file_addr = "/1/2012-02-22-15.gz"

In [215]:
# The inner text of <doc> is stored as the None tag where tag.string != '\n' (is more than just a newline character
# .string of <doc>'s inner text parses inner <a> tags etc. into just their inner text

# open gz file and return html
def get_html_from_gz(addr):
    html = None
    with gzip.open(addr) as f:
        print(addr)
        html = bs(f)
    return html


# parse html and return an ordered 2d list [doc:fields]
# parse html and add to orded 2d corpus list [doc:fields]
def get_docs(html, corpus_list):
    for d in html.find_all("doc"):
        doc = OrderedDict.fromkeys(tags)
        for c in d.children:  # children use direct children, descedants uses all
            if c.name is None and c.string != '\n':  # inner body of <doc> tag
                doc['text'] = c.string
            if c.name in doc:
                doc[c.name] = c.string
            elif c.name is not None:
                print("doc has unexpected field: " + str(c.name))
                print("parent field: " + c.parent.name)
                print("field content: " + str(c))
        
        missing_fields = [k for (k,v) in doc.items() if v == None]
        for m in missing_fields:
            print("Doc is missing field: " + m)
            
        corpus_list.append(list(doc.values()))
        
            


# recursively find gz html files from a directory address
def search_dir(path, gz_paths):    
    # separate the subdirectories and html files 
    # (help maintain sequential order of insertion)
    subdirs = []
#     dir_gz_paths = []
    for f in os.scandir(path):
        if f.is_dir():
            subdirs.append(f.path)
        elif os.path.splitext(f.path)[-1].lower() == ".gz":
            gz_paths.append(f.path)
    
    # search subdirs
    for sd in subdirs:
        search_dir(sd, gz_paths)
#     gz_paths.append(dir_gz_paths) # add found gz files after sub dirs searched


# load all formatted gzipped html files into dataframe
def load_corpus(path):
    # collect all html gz files from path and all sub dirs
    gz_paths = []
    search_dir(path, gz_paths)
    
    # process all documents into formatted list
    print("Opening files...")
    corpus_list = []
    for p in gz_paths:
        get_docs(get_html_from_gz(p), corpus_list)
    
    # return and convert list into DataFrame
    return pd.DataFrame(corpus_list, columns=tags)
    
        
# load corpus as DataFrame
corpus = load_corpus(corpus_path)
print("Corpus loaded into DataFrame")
        

Opening files...
/nfs/trects-kba2014-filtered/9/2012-11-10-17.gz
/nfs/trects-kba2014-filtered/9/2012-11-14-16.gz
/nfs/trects-kba2014-filtered/9/2012-11-15-01.gz
/nfs/trects-kba2014-filtered/9/2012-11-12-17.gz
/nfs/trects-kba2014-filtered/9/2012-11-12-18.gz
/nfs/trects-kba2014-filtered/9/2012-11-10-23.gz
/nfs/trects-kba2014-filtered/9/2012-11-15-04.gz
/nfs/trects-kba2014-filtered/9/2012-11-14-23.gz
/nfs/trects-kba2014-filtered/9/2012-11-08-16.gz
/nfs/trects-kba2014-filtered/9/2012-11-15-07.gz
/nfs/trects-kba2014-filtered/9/2012-11-08-03.gz
/nfs/trects-kba2014-filtered/9/2012-11-15-22.gz
/nfs/trects-kba2014-filtered/9/2012-11-15-18.gz
/nfs/trects-kba2014-filtered/9/2012-11-17-08.gz
/nfs/trects-kba2014-filtered/9/2012-11-09-04.gz
/nfs/trects-kba2014-filtered/9/2012-11-14-22.gz
/nfs/trects-kba2014-filtered/9/2012-11-13-06.gz
/nfs/trects-kba2014-filtered/9/2012-11-08-06.gz
/nfs/trects-kba2014-filtered/9/2012-11-09-06.gz
/nfs/trects-kba2014-filtered/9/2012-11-08-22.gz
/nfs/trects-kba2014-fil

/nfs/trects-kba2014-filtered/9/2012-11-16-19.gz
/nfs/trects-kba2014-filtered/9/2012-11-08-11.gz
/nfs/trects-kba2014-filtered/9/2012-11-08-10.gz
/nfs/trects-kba2014-filtered/9/2012-11-13-10.gz
/nfs/trects-kba2014-filtered/9/2012-11-07-21.gz
/nfs/trects-kba2014-filtered/9/2012-11-14-15.gz
/nfs/trects-kba2014-filtered/9/2012-11-09-17.gz
/nfs/trects-kba2014-filtered/9/2012-11-09-20.gz
/nfs/trects-kba2014-filtered/9/2012-11-11-22.gz
/nfs/trects-kba2014-filtered/9/2012-11-17-01.gz
/nfs/trects-kba2014-filtered/9/2012-11-16-01.gz
/nfs/trects-kba2014-filtered/9/2012-11-08-12.gz
/nfs/trects-kba2014-filtered/9/2012-11-10-12.gz
/nfs/trects-kba2014-filtered/9/2012-11-15-16.gz
/nfs/trects-kba2014-filtered/9/2012-11-14-19.gz
/nfs/trects-kba2014-filtered/9/2012-11-16-05.gz
/nfs/trects-kba2014-filtered/9/2012-11-10-14.gz
/nfs/trects-kba2014-filtered/9/2012-11-12-12.gz
/nfs/trects-kba2014-filtered/9/2012-11-13-21.gz
/nfs/trects-kba2014-filtered/9/2012-11-13-20.gz
/nfs/trects-kba2014-filtered/9/2012-11-0

/nfs/trects-kba2014-filtered/5/2012-08-30-11.gz
/nfs/trects-kba2014-filtered/5/2012-09-01-02.gz
/nfs/trects-kba2014-filtered/5/2012-09-05-13.gz
/nfs/trects-kba2014-filtered/5/2012-09-05-20.gz
/nfs/trects-kba2014-filtered/5/2012-09-01-10.gz
/nfs/trects-kba2014-filtered/5/2012-09-06-09.gz
/nfs/trects-kba2014-filtered/5/2012-09-06-17.gz
/nfs/trects-kba2014-filtered/5/2012-09-01-13.gz
/nfs/trects-kba2014-filtered/5/2012-08-29-17.gz
/nfs/trects-kba2014-filtered/5/2012-09-03-18.gz
/nfs/trects-kba2014-filtered/5/2012-09-06-12.gz
/nfs/trects-kba2014-filtered/5/2012-09-04-22.gz
/nfs/trects-kba2014-filtered/5/2012-09-03-07.gz
/nfs/trects-kba2014-filtered/5/2012-08-29-12.gz
/nfs/trects-kba2014-filtered/5/2012-09-05-07.gz
/nfs/trects-kba2014-filtered/5/2012-09-02-05.gz
/nfs/trects-kba2014-filtered/5/2012-09-06-02.gz
/nfs/trects-kba2014-filtered/5/2012-08-29-21.gz
/nfs/trects-kba2014-filtered/5/2012-09-04-11.gz
/nfs/trects-kba2014-filtered/5/2012-08-29-16.gz
/nfs/trects-kba2014-filtered/5/2012-09-0

/nfs/trects-kba2014-filtered/1/2012-02-27-04.gz
/nfs/trects-kba2014-filtered/1/2012-02-24-12.gz
/nfs/trects-kba2014-filtered/1/2012-02-28-02.gz
/nfs/trects-kba2014-filtered/1/2012-02-25-17.gz
/nfs/trects-kba2014-filtered/1/2012-02-27-23.gz
/nfs/trects-kba2014-filtered/1/2012-02-28-03.gz
/nfs/trects-kba2014-filtered/1/2012-03-02-22.gz
/nfs/trects-kba2014-filtered/1/2012-03-03-07.gz
/nfs/trects-kba2014-filtered/1/2012-02-25-08.gz
/nfs/trects-kba2014-filtered/1/2012-02-27-16.gz
/nfs/trects-kba2014-filtered/1/2012-02-26-09.gz
/nfs/trects-kba2014-filtered/1/2012-02-24-00.gz
/nfs/trects-kba2014-filtered/1/2012-03-01-13.gz
/nfs/trects-kba2014-filtered/1/2012-02-27-21.gz
/nfs/trects-kba2014-filtered/1/2012-02-23-00.gz
/nfs/trects-kba2014-filtered/1/2012-03-02-10.gz
/nfs/trects-kba2014-filtered/1/2012-02-22-17.gz
/nfs/trects-kba2014-filtered/1/2012-02-22-13.gz
/nfs/trects-kba2014-filtered/1/2012-02-29-00.gz
/nfs/trects-kba2014-filtered/1/2012-02-27-10.gz
/nfs/trects-kba2014-filtered/1/2012-02-2

/nfs/trects-kba2014-filtered/1/2012-02-25-01.gz
/nfs/trects-kba2014-filtered/1/2012-03-03-10.gz
/nfs/trects-kba2014-filtered/1/2012-03-01-12.gz
/nfs/trects-kba2014-filtered/1/2012-02-28-14.gz
/nfs/trects-kba2014-filtered/1/2012-02-28-20.gz
/nfs/trects-kba2014-filtered/1/2012-02-24-02.gz
/nfs/trects-kba2014-filtered/1/2012-02-25-03.gz
/nfs/trects-kba2014-filtered/1/2012-02-25-00.gz
/nfs/trects-kba2014-filtered/1/2012-02-28-00.gz
/nfs/trects-kba2014-filtered/1/2012-03-03-00.gz
/nfs/trects-kba2014-filtered/1/2012-02-23-02.gz
/nfs/trects-kba2014-filtered/1/2012-02-25-21.gz
/nfs/trects-kba2014-filtered/1/2012-03-02-06.gz
/nfs/trects-kba2014-filtered/1/2012-02-28-19.gz
/nfs/trects-kba2014-filtered/1/2012-02-29-15.gz
/nfs/trects-kba2014-filtered/1/2012-02-24-20.gz
/nfs/trects-kba2014-filtered/1/2012-03-03-08.gz
/nfs/trects-kba2014-filtered/1/2012-03-01-18.gz
/nfs/trects-kba2014-filtered/1/2012-02-29-11.gz
/nfs/trects-kba2014-filtered/1/2012-03-01-10.gz
/nfs/trects-kba2014-filtered/1/2012-03-0

...

/nfs/trects-kba2014-filtered/2/2012-09-14-23.gz
/nfs/trects-kba2014-filtered/2/2012-09-20-21.gz
/nfs/trects-kba2014-filtered/2/2012-09-16-02.gz
/nfs/trects-kba2014-filtered/2/2012-09-18-08.gz
/nfs/trects-kba2014-filtered/2/2012-09-18-02.gz
/nfs/trects-kba2014-filtered/2/2012-09-19-06.gz
/nfs/trects-kba2014-filtered/2/2012-09-21-02.gz
/nfs/trects-kba2014-filtered/2/2012-09-20-20.gz
/nfs/trects-kba2014-filtered/2/2012-09-15-23.gz
/nfs/trects-kba2014-filtered/2/2012-09-18-12.gz
/nfs/trects-kba2014-filtered/2/2012-09-16-13.gz
/nfs/trects-kba2014-filtered/2/2012-09-21-06.gz
/nfs/trects-kba2014-filtered/2/2012-09-14-15.gz
/nfs/trects-kba2014-filtered/2/2012-09-18-13.gz
/nfs/trects-kba2014-filtered/2/2012-09-15-10.gz
/nfs/trects-kba2014-filtered/2/2012-09-15-22.gz
/nfs/trects-kba2014-filtered/2/2012-09-13-04.gz
/nfs/trects-kba2014-filtered/2/2012-09-15-21.gz
/nfs/trects-kba2014-filtered/2/2012-09-12-09.gz
/nfs/trects-kba2014-filtered/2/2012-09-16-05.gz
/nfs/trects-kba2014-filtered/2/2012-09-1

/nfs/trects-kba2014-filtered/3/2012-07-21-01.gz
/nfs/trects-kba2014-filtered/3/2012-07-21-20.gz
/nfs/trects-kba2014-filtered/3/2012-07-28-07.gz
/nfs/trects-kba2014-filtered/3/2012-07-25-07.gz
/nfs/trects-kba2014-filtered/3/2012-07-20-15.gz
/nfs/trects-kba2014-filtered/3/2012-07-24-14.gz
/nfs/trects-kba2014-filtered/3/2012-07-21-22.gz
/nfs/trects-kba2014-filtered/3/2012-07-26-05.gz
/nfs/trects-kba2014-filtered/3/2012-07-22-21.gz
/nfs/trects-kba2014-filtered/3/2012-07-20-11.gz
/nfs/trects-kba2014-filtered/3/2012-07-21-06.gz
/nfs/trects-kba2014-filtered/3/2012-07-27-02.gz
/nfs/trects-kba2014-filtered/3/2012-07-28-05.gz
/nfs/trects-kba2014-filtered/3/2012-07-29-12.gz
/nfs/trects-kba2014-filtered/3/2012-07-23-15.gz
/nfs/trects-kba2014-filtered/3/2012-07-24-23.gz
/nfs/trects-kba2014-filtered/3/2012-07-27-05.gz
/nfs/trects-kba2014-filtered/3/2012-07-24-17.gz
/nfs/trects-kba2014-filtered/3/2012-07-21-07.gz
/nfs/trects-kba2014-filtered/3/2012-07-27-15.gz
/nfs/trects-kba2014-filtered/3/2012-07-2

/nfs/trects-kba2014-filtered/3/2012-07-30-01.gz
/nfs/trects-kba2014-filtered/3/2012-07-20-13.gz
/nfs/trects-kba2014-filtered/3/2012-07-21-14.gz
/nfs/trects-kba2014-filtered/3/2012-07-26-14.gz
/nfs/trects-kba2014-filtered/3/2012-07-29-23.gz
/nfs/trects-kba2014-filtered/3/2012-07-26-16.gz
/nfs/trects-kba2014-filtered/3/2012-07-25-13.gz
/nfs/trects-kba2014-filtered/3/2012-07-26-21.gz
/nfs/trects-kba2014-filtered/3/2012-07-24-12.gz
/nfs/trects-kba2014-filtered/3/2012-07-23-10.gz
/nfs/trects-kba2014-filtered/3/2012-07-30-00.gz
/nfs/trects-kba2014-filtered/3/2012-07-24-00.gz
/nfs/trects-kba2014-filtered/3/2012-07-21-11.gz
/nfs/trects-kba2014-filtered/3/2012-07-25-04.gz
/nfs/trects-kba2014-filtered/3/2012-07-30-03.gz
/nfs/trects-kba2014-filtered/3/2012-07-30-04.gz
/nfs/trects-kba2014-filtered/3/2012-07-24-15.gz
/nfs/trects-kba2014-filtered/3/2012-07-22-08.gz
/nfs/trects-kba2014-filtered/3/2012-07-29-08.gz
/nfs/trects-kba2014-filtered/3/2012-07-25-06.gz
/nfs/trects-kba2014-filtered/3/2012-07-2

/nfs/trects-kba2014-filtered/10/2012-11-26-20.gz
/nfs/trects-kba2014-filtered/10/2012-11-26-09.gz
/nfs/trects-kba2014-filtered/10/2012-11-24-04.gz
/nfs/trects-kba2014-filtered/10/2012-11-24-00.gz
/nfs/trects-kba2014-filtered/10/2012-11-23-05.gz
/nfs/trects-kba2014-filtered/10/2012-11-27-08.gz
/nfs/trects-kba2014-filtered/10/2012-11-24-16.gz
/nfs/trects-kba2014-filtered/10/2012-11-27-21.gz
/nfs/trects-kba2014-filtered/10/2012-11-29-09.gz
/nfs/trects-kba2014-filtered/10/2012-11-28-21.gz
/nfs/trects-kba2014-filtered/10/2012-11-27-00.gz
/nfs/trects-kba2014-filtered/10/2012-11-27-09.gz
/nfs/trects-kba2014-filtered/10/2012-11-26-18.gz
/nfs/trects-kba2014-filtered/10/2012-11-29-22.gz
/nfs/trects-kba2014-filtered/10/2012-11-26-00.gz
/nfs/trects-kba2014-filtered/10/2012-11-26-06.gz
/nfs/trects-kba2014-filtered/10/2012-11-25-04.gz
/nfs/trects-kba2014-filtered/10/2012-11-22-19.gz
/nfs/trects-kba2014-filtered/10/2012-11-25-09.gz
/nfs/trects-kba2014-filtered/10/2012-11-21-14.gz
/nfs/trects-kba2014-

/nfs/trects-kba2014-filtered/10/2012-11-24-19.gz
/nfs/trects-kba2014-filtered/10/2012-11-25-10.gz
/nfs/trects-kba2014-filtered/10/2012-11-28-03.gz
/nfs/trects-kba2014-filtered/10/2012-11-26-11.gz
/nfs/trects-kba2014-filtered/10/2012-11-29-14.gz
/nfs/trects-kba2014-filtered/10/2012-11-27-01.gz
/nfs/trects-kba2014-filtered/10/2012-12-01-10.gz
/nfs/trects-kba2014-filtered/10/2012-11-29-15.gz
/nfs/trects-kba2014-filtered/10/2012-11-30-03.gz
/nfs/trects-kba2014-filtered/10/2012-11-21-23.gz
/nfs/trects-kba2014-filtered/10/2012-11-24-18.gz
/nfs/trects-kba2014-filtered/10/2012-11-21-15.gz
/nfs/trects-kba2014-filtered/10/2012-11-24-05.gz
/nfs/trects-kba2014-filtered/10/2012-11-21-20.gz
/nfs/trects-kba2014-filtered/10/2012-11-21-16.gz
/nfs/trects-kba2014-filtered/10/2012-11-22-15.gz
/nfs/trects-kba2014-filtered/10/2012-11-21-21.gz
/nfs/trects-kba2014-filtered/10/2012-11-24-09.gz
/nfs/trects-kba2014-filtered/10/2012-11-21-12.gz
/nfs/trects-kba2014-filtered/10/2012-11-23-22.gz
/nfs/trects-kba2014-

In [213]:
# The inner text of <doc> is stored as the None tag where tag.string != '\n' (is more than just a newline character)
# .string of <doc>'s inner text parses inner <a> tags etc. into just their inner text

# def get_docs_2(html, corpus_list):
#     for d in html.find_all("doc"):
#         doc = OrderedDict.fromkeys(tags)
#         for c in d.children:  # children use direct children, descedants uses all
#             if c.name in doc:
#                 doc[c.name] = c.string
#             elif c.name is not None:
#                 print("doc has unexpected field: " + str(c.name))
#                 print("parent field: " + c.parent.name)
#                 print("field content: " + str(c))
        
#         missing_fields = [k for (k,v) in doc.items() if v == None]
#         for m in missing_fields:
#             print("Doc is missing field: " + m)
            
#         corpus_list.append(list(doc.values()))
        
# # Try looping through using next_sibling, adding each tag content into proper field, then adding doc body

# def get_docs_3(html, corpus_list):
#     for d in html.find_all("doc"):
# #        doc = OrderedDict.fromkeys(tags)
#         doc = ""
#         a_tags_test = []
#         for c in d.descendants:  # children was working, changed to descendants to test inner <a> tags
#             if c.name is not None and c.name not in tags:
#                 print(c.name)
            
# #             if c.name is "a":
# # #                 print("a tag:")
# # #                 print(c)
# # #                 print("")
# #                 if len(c.contents) > 0:
# #                     a_tags_test.append(c.contents[0])
# #             elif c.name is None and c.string != '\n':
# #                 doc = c.string
# #         for a_test in a_tags_test:
# #             print("looking for: " + a_test)
# #             if a_test in c.string:
# #                 print("found")
# #             else:
# #                 print("not found")
# #             print("")
# # #             if c.name is None:
# # #                 if c.string != '\n':
# # #                     print(c.string)
# # #                     print("printed")
# #                     print("")
#     print("finished get_docs_3")

In [212]:
# missing_field_doc = "/nfs/trects-kba2014-filtered/2/2012-09-12-10.gz"
# miss_list = []
# get_docs_3(get_html_from_gz(missing_field_doc), miss_list)

In [216]:
print(corpus.head(10))

                                      streamid  \
0  1352566975-da4262fddc1eebbb4e88013b479f5e3d   
1  1352344307-8c433f296685f56ef22fc65daea87905   
2  1352344533-65f8f2ab767eb611b571d97732198c18   
3  1352345570-4c3eb77a086a0cbff460b4dc36ba826d   
4  1352347119-fe405af03103645fd87eb247eceb1c99   
5  1352346755-0721de8fe5aa04c5c2f6888ee112104a   
6  1352346862-fe405af03103645fd87eb247eceb1c99   
7  1352344686-4c3eb77a086a0cbff460b4dc36ba826d   
8  1352344188-919a40913daee20c9fa6225b773feeab   
9  1352346779-43eaec0211a7c916614328bf5da73933   

                              docid     yyyymmddhh        kbastream  \
0  da4262fddc1eebbb4e88013b479f5e3d  2012-11-10-17           WEBLOG   
1  8c433f296685f56ef22fc65daea87905  2012-11-08-03  MAINSTREAM_NEWS   
2  65f8f2ab767eb611b571d97732198c18  2012-11-08-03  MAINSTREAM_NEWS   
3  4c3eb77a086a0cbff460b4dc36ba826d  2012-11-08-03  MAINSTREAM_NEWS   
4  fe405af03103645fd87eb247eceb1c99  2012-11-08-03  MAINSTREAM_NEWS   
5  0721de8fe5aa04c5c2f6

In [221]:
print(corpus.iloc[6,:])

streamid            1352346862-fe405af03103645fd87eb247eceb1c99
docid                          fe405af03103645fd87eb247eceb1c99
yyyymmddhh                                        2012-11-08-03
kbastream                                       MAINSTREAM_NEWS
zulu                                     2012-11-08T03:54:22.0Z
epoch                                                1352346862
title         Guatemala Earthquake: At Least 48 Dead - Yahoo...
text          \nGuatemala Earthquake: At Least 48 Dead - Yah...
url           http://uk.news.yahoo.com/guatemala-earthquake-...
Name: 6, dtype: object
